# Load configuration

In [1]:
import os
import yaml


with open("../conf/service.dev.yaml", 'r') as f:
    configs = yaml.safe_load(f)
os.environ['OPENAI_API_KEY'] = configs['openai']['api_key']

# 1. Prepare agent components

## 1.1 Build prompt

In [2]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.format_scratchpad import format_to_openai_functions


prompt = RunnablePassthrough.assign(
    agent_scratchpad=lambda x: format_to_openai_functions(x['intermediate_steps'])
) | ChatPromptTemplate.from_messages([
    ('system', "You are helpful but sassy assistant. Please think carefully and step by step then answer with korean."),
    MessagesPlaceholder(variable_name='chat_history'),
    ('user', "{input}"),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

## 1.2 Build output parser

In [3]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

parser = OpenAIFunctionsAgentOutputParser()

## 1.3 Build model

### 1.3.1 Build tools

In [4]:
from pydantic import BaseModel, Field
from langchain.agents import tool
import wikipedia
from wikipedia.exceptions import PageError, DisambiguationError
from langchain_core.utils.function_calling import convert_to_openai_function


class SearchWikipediaInput(BaseModel):
    query: str = Field(description='Query to search for in Wikipedia')

@tool(args_schema=SearchWikipediaInput)
def search_wikipedia(query: str) -> str:
    """Do a Wikipedia search for `query`."""
    titles = wikipedia.search(query)
    summaries = []

    for title in titles:
        try:
            page = wikipedia.page(title=title, auto_suggest=False)
            summary = page.summary
            summaries.append(f"{title}: {summary}")
            break
        except (PageError, DisambiguationError):
            pass
    
    if not summaries:
        return "No good Wikipedia search result was found."
    
    return '\n\n'.join(summaries)

functions = [convert_to_openai_function(search_wikipedia)]

### 1.3.2 Build model

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).bind(
    functions=functions
)

## 1.4 Build chain

In [6]:
chain = prompt | model | parser

## 1.5 Build memory

In [7]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 2. Build agent

In [8]:
from langchain.agents import AgentExecutor

agent = AgentExecutor(agent=chain, tools=[search_wikipedia], verbose=True, memory=memory)

# 3. Invoking

In [9]:
agent.invoke({'input': "안녕. 난 지민이라고 해. 네 이름은 뭐야?"});



> Entering new AgentExecutor chain...
안녕 지민이! 나는 도움이 되는데, 내 이름은 없어. 어떻게 도와줄까?

> Finished chain.


In [10]:
agent.invoke({'input': "몇 가지 궁금한 점이 있는데"});



> Entering new AgentExecutor chain...
물어봐! 궁금한 점이 무엇이야? 어떤 도움을 원하니?

> Finished chain.


In [11]:
agent.invoke({'input': "머신러닝과 관련된 키워드를 몇 개 선택해줘. 그걸 기반으로 공부하고 싶거든."});



> Entering new AgentExecutor chain...
물론이야! 머신러닝과 관련된 몇 가지 키워드를 알려줄게. 이 키워드들을 기반으로 공부하면 좋을 거야:

1. 지도 학습 (Supervised Learning)
2. 비지도 학습 (Unsupervised Learning)
3. 강화 학습 (Reinforcement Learning)
4. 신경망 (Neural Networks)
5. 딥러닝 (Deep Learning)
6. 데이터 전처리 (Data Preprocessing)
7. 특성 추출 (Feature Extraction)
8. 모델 평가 (Model Evaluation)
9. 군집화 (Clustering)
10. 회귀 (Regression)

이 키워드들을 참고하여 머신러닝에 대해 공부해보면 좋을 거야. 어떤 키워드에 대해 더 자세히 알고 싶어?

> Finished chain.


In [12]:
agent.invoke({'input': "너가 알려준 키워드들 중에 상위 3가지에 대해 wikipedia 검색을 사용해서 자세히 설명해줘. 영어로 검색해봐야할거야."});



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Supervised Learning'}`


Supervised learning: Supervised learning (SL) is a paradigm in machine learning where input objects (for example, a vector of predictor variables) and a desired output value (also known as human-labeled supervisory signal) train a model. The training data is processed, building a function that maps new data on expected output values.  An optimal scenario will allow for the algorithm to correctly determine output values for unseen instances. This requires the learning algorithm to generalize from the training data to unseen situations in a "reasonable" way (see inductive bias). This statistical quality of an algorithm is measured through the so-called generalization error.


Invoking: `search_wikipedia` with `{'query': 'Unsupervised Learning'}`


Unsupervised learning: Unsupervised learning is a method in machine learning where, in contrast to supervised learning, algorithms l

In [13]:
agent.invoke({'input': "Thank you bro."});



> Entering new AgentExecutor chain...
You're welcome, bro! If you have any more questions, feel free to ask. I'm here to help!

> Finished chain.
